In [ ]:
from openai import OpenAI
import pandas as pd

# Point to the LM Studio local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")
 

with open(r'GPT\data\gpt_data.csv', 'r', encoding='utf-8', errors='replace') as file:
    df = pd.read_csv(file)


def clean_output(output):
    """Remove 'None' from the output string."""
    if isinstance(output, str):
        return output.replace("None", "").strip()  # Remove 'None' and trim whitespace
    return output

df['LLM Output']=None

for i, row in df.iterrows(): 
    text = str(row['Incident Narrative'])
    message = [
        {"role": "system", "content": """Your task is to classify each news article into one of the following categories. The output should only contain the category name, with no explanation and other output:
    Unlawful detention
    Human trafficking
    Enslavement
    Willful killing of civilians
    Mass execution
    Kidnapping
    Extrajudicial killing
    Forced disappearance
    Damage or destruction of civilian critical infrastructure
    Damage or destruction, looting, or theft of cultural heritage
    Military operations (battle, shelling)
    Gender-based or other conflict-related sexual violence
    Violent crackdowns on protesters/opponents/civil rights abuse
    Indiscriminate use of weapons
    Torture or indications of torture
    Persecution based on political, racial, ethnic, gender, or sexual orientation
    Movement of military, paramilitary, or other troops and equipment

    Classify each news article into one of the above categories. Note that a single article can be classified into only one category based on the most prominent theme. If an article fits into multiple categories, select the one that best describes the primary issue.
    
    """},
        {"role": "user", "content": text},
    ]
    # Call API to generate responses
    completion = client.chat.completions.create(
        model="model-identifier",
        messages=message,
        temperature=0,
        stream=True,
    )
    output =''
    for chunk in completion:
        output += str(chunk.choices[0].delta.content)
    df.at[i,'LLM Output'] = output
    
df['LLM Output'] = df['LLM Output'].apply(clean_output)

df.to_csv('Sudan_Conflict_SLC_Zeroshot.csv', index=False, encoding='utf-8')
